In [3]:
from benchmark import Benchmark
cogent_benchmark = Benchmark("pycogent")

In [4]:
@cogent_benchmark
def object_creation():
    pass

The slowest run took 44.70 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 90.7 ns per loop
